# 구글 플레이스토어 웹 리뷰 크롤러

## 초기세팅

### 1. 라이브러리 설치하기

In [4]:
!pip install selenium
!pip install pandas

### 2. Headless 설정
- 프로그램 실행 시 chrome 브라우저 창을 띄우지 않게 옵션을 설정할 수 있다.
- 일부 사이트에서 headless 접근을 차단하기 때문에 user-agent 설정이 필요하다.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# headless
options = webdriver.ChromeOptions()
#options.add_argument("--headless=new")
options.add_argument('window-size=1920x1080')
# user agent
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36")
options.add_argument("disable-gpu")

In [4]:
# chromedriver
driver = webdriver.Chrome(options=options)

### 3. URL 설정
- 현재 토스 구글플레이 페이지로 설정되어있다.

In [5]:
URL = "https://play.google.com/store/apps/details?id=viva.republica.toss&hl=ko&gl=US"

# Load Page
# chrome을 띄워 토스 구글플레이 페이지를 킨다.
driver.get(url=URL)

## '리뷰 모두 보기' 버튼 클릭

In [6]:
import time

# 최하단으로 스크롤내리기
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# 리뷰 모두보기 클릭
spread_review = driver.find_element(by=By.XPATH, value = '/html/body/c-wiz[2]/div/div/div[1]/div/div[2]/div/div[1]/div[1]/c-wiz[5]/section/div/div[2]/div[5]/div/div/button')
isTrue = spread_review.is_displayed()
if isTrue :
    driver.execute_script("arguments[0].click();", spread_review)
    # spread_review.click()
    time.sleep(1.5)

## 무한스크롤

In [7]:
# 리뷰 팝업창 element
all_reviews = driver.find_element(by=By.XPATH, value ='/html/body/div[4]/div[2]/div/div/div/div/div[2]')
# 현재 리뷰 팝업창 height 구하기
last_height = driver.execute_script("return arguments[0].scrollHeight", all_reviews)
# last_height = driver.execute_script("return document.getElementsByClassName('odk6He')[0].scrollHeight")

while True :
    # 현재 height만큼 스크롤
    driver.execute_script(f'arguments[0].scrollTop = {last_height}', all_reviews)
    time.sleep(1)   # 로드 기다리는 시간

    new_height = driver.execute_script("return arguments[0].scrollHeight", all_reviews)
    # new_height = driver.execute_script("return document.getElementsByClassName('odk6He')[0].scrollHeight")

    # 스크롤을 진행했는데도 이전 height와 스크롤 이후 height가 같으면 더이상 로드할 데이터가 없다는 뜻이므로 스크롤 중단
    if new_height == last_height :
        break

    last_height = new_height

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.106)
Stacktrace:
	GetHandleVerifier [0x00007FF6F74C7072+63090]
	(No symbol) [0x00007FF6F7432CC2]
	(No symbol) [0x00007FF6F72CEC65]
	(No symbol) [0x00007FF6F72ACA7C]
	(No symbol) [0x00007FF6F733D687]
	(No symbol) [0x00007FF6F7352AC1]
	(No symbol) [0x00007FF6F7336D83]
	(No symbol) [0x00007FF6F73083A8]
	(No symbol) [0x00007FF6F7309441]
	GetHandleVerifier [0x00007FF6F78C25CD+4238285]
	GetHandleVerifier [0x00007FF6F78FF72D+4488493]
	GetHandleVerifier [0x00007FF6F78F7A0F+4456463]
	GetHandleVerifier [0x00007FF6F75A05B6+953270]
	(No symbol) [0x00007FF6F743E58F]
	(No symbol) [0x00007FF6F7439264]
	(No symbol) [0x00007FF6F743939B]
	(No symbol) [0x00007FF6F7429BD4]
	BaseThreadInitThunk [0x00007FFDE7CC7344+20]
	RtlUserThreadStart [0x00007FFDE8AC26B1+33]


## 데이터 수집

In [ ]:
import pandas as pd

data = pd.DataFrame(data=[], columns=['날짜','리뷰','별점'])
#날짜, 리뷰, 별점 수집
dates = driver.find_elements(by=By.XPATH, value = '//div[@class="odk6He"]//span[@class="bp9Aid"]')
reviews=driver.find_elements(by=By.XPATH, value = '//div[@class="odk6He"]//div[@class="h3YV2d"]')
stargrades = driver.find_elements(by=By.XPATH, value = '//div[@class="odk6He"]//div[@class="iXRFPc"]')


# 수집한 리뷰를 DataFrame에 삽입
for i in range(len(reviews)):
    tmp = []
    tmp.append(dates[i].text)
    tmp.append(reviews[i].text)
    tmp.append(stargrades[i].get_attribute('aria-label'))

    tmp = pd.DataFrame(data=[tmp], columns = data.columns)
    data = pd.concat([data,tmp])

# 인덱스 재배열
data.reset_index(inplace=True, drop=True)

## 별점 데이터 전처리

In [ ]:
import re

# 별점에서 숫자만 추출
data['별점'] = data['별점'].apply(lambda x : x[5:])
m = re.compile('[0-9][\.0-9]*') # 정규표현식
data['별점'] = data['별점'].apply(lambda x : m.findall(x)[0])

In [ ]:
# 엑셀 파일로 추출
data.to_excel('reviews.xlsx')
driver.close()